In [ ]:
import torch
import lightning.pytorch as pl

from mymodels import Model_Wrapper, Preprocess
from myutils import View
from yolo_wrapper import YOLOv8_COCO_Wrapper
from coco_ds import CocoResults

import logging
logging.disable(logging.INFO)

In [ ]:
# configs to speed up training
torch.set_float32_matmul_precision('medium')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [ ]:
PATH_TO_COCO = '/f_storage/datasets/coco/2017'
NUM_IMG_EVAL = 10000
prep = Preprocess(PATH_TO_COCO, (640, 640), dataset_type='coco', shuffle=False, batch_size=64)
%matplotlib inline

In [ ]:
prep.reset_trans()
coco = prep.get_loader()
model = YOLOv8_COCO_Wrapper(CocoResults(coco.dataset), 'yolov8m.pt')
model = Model_Wrapper(model, model_type='yolo')
# trainer = pl.Trainer(accelerator='auto', limit_test_batches=10)
trainer = pl.Trainer(accelerator='auto')

In [ ]:
def compare_ds(model, dataloaders, labels, preview_idxs:list, view_only=False, show_bbox=True, figsize=(12,6), verbose=True, skip_results:list=[0]):
    results = {}
    samples = []
    #display first
    for ds in dataloaders:
        sample = torch.stack([ds.dataset[i][1] for i in preview_idxs])
        
        if show_bbox:
            samples.append(model.plot(sample))
        else:
            samples.append(sample)
        
    View.compare_color(samples, labels, figsize=figsize)

    
    if not view_only:
        for i in range(len(dataloaders)):
            if i in skip_results:
                continue
            
            result = trainer.test(model, dataloaders[i], verbose=verbose)
            results[labels[i]] = result
            
            
        return results
    else:
        return None

##### Test On Orignal Dataset

In [ ]:
preview_idxs = [2, 53, 300, 607, 3584]
samples  = torch.stack([coco.dataset[i][1] for i in preview_idxs])
plots = model.plot(samples)
View.compare_color(plots, 'Original')
# View.histogram(samples)
# orig_results = trainer.test(model, coco)

##### Test filters on Orignal Dataset

In [ ]:
prep.reset_trans()
coco = prep.get_loader()
coco_histeq = prep.copy().hist_eq().get_loader()
coco_retinex = prep.copy().retinex('SSR', 100).get_loader()
coco_results = compare_ds(model, 
                                 [coco, coco_histeq, coco_retinex],
                                 ['Original', 'Original + HistEQ', 'Original + Retinex'], 
                                 preview_idxs,
                                 view_only=False)

##### Test on Dark Images

In [ ]:
prep.reset_trans()
coco_dark = prep.luminance(1/8).get_loader()
coco_dark_histeq = prep.copy().hist_eq().get_loader()
coco_dark_retinex = prep.copy().retinex('SSR', 100).get_loader()
coco_dark_results = compare_ds(model, 
                                 [prep.basic_loader(), coco_dark, coco_dark_histeq, coco_dark_retinex],
                                 ['Original', 'Dark', 'Dark + HistEQ', 'Dark + Retinex'], 
                                 preview_idxs,
                                 view_only=False)

##### Test on Over Exposed Images

In [ ]:
prep.reset_trans()
coco_bright = prep.luminance(2).get_loader()
coco_bright_histeq = prep.copy().hist_eq().get_loader()
coco_bright_retinex = prep.copy().retinex('SSR', 100).get_loader()
coco_bright_results = compare_ds(model, 
                                 [prep.basic_loader(), coco_bright, coco_bright_histeq, coco_bright_retinex],
                                 ['Original', 'Bright', 'Bright + HistEQ', 'Bright + Retinex'], 
                                 preview_idxs)

##### Test on hazy images

In [ ]:
prep.reset_trans()
coco_mod = prep.fog(10).get_loader()
coco_mod_histeq = prep.copy().hist_eq().get_loader()
coco_mod_retinex = prep.copy().retinex('SSR', 100).get_loader()
coco_mod_results = compare_ds(model, 
                                 [prep.basic_loader(), coco_mod, coco_mod_histeq, coco_mod_retinex],
                                 ['Original', 'Hazy / Foggy', 'Hazy + HistEQ', 'Hazy + Retinex'], 
                                 preview_idxs,
                                 view_only=False)

##### Test on Dark Rainy Images

In [ ]:
prep.reset_trans()
coco_mod = prep.rain(2, 75).get_loader()
coco_mod_histeq = prep.copy().hist_eq().get_loader()
coco_mod_retinex = prep.copy().retinex('SSR', 100).get_loader()
coco_mod_results = compare_ds(model, 
                                 [prep.basic_loader(), coco_mod, coco_mod_histeq, coco_mod_retinex],
                                 ['Original', 'Dark Rainy', 'Dark Rainy+ HistEQ', 'Dark Rainy + Retinex'], 
                                 preview_idxs,
                                 view_only=False)

##### Test Histogram Eq on Low Contrast Images

In [ ]:
# prep.reset_trans()
# coco_low_cont = prep.brightness_contrast(0,5).get_loader()
# coco_low_cont_histeq = prep.copy().hist_eq().get_loader()
# coco_low_cont_retinex = prep.copy().retinex('SSR', 100).get_loader()
# coco_low_cont_results = compare_ds(model, 
#                                  [prep.basic_loader(), coco_low_cont, coco_low_cont_histeq, coco_low_cont_retinex],
#                                  ['Original', 'Low Contrast', 'Low Contrast + HistEQ', 'Low Contrast + Retinex'], 
#                                  preview_idxs)

##### Test Remapping Distribution to be low contrast

In [ ]:
# prep.reset_trans()
# coco_mod = prep.dist_remap(20/255,20/255).get_loader()
# coco_mod_histeq = prep.copy().hist_eq().get_loader()
# coco_mod_retinex = prep.copy().retinex('SSR', 100).get_loader()
# coco_mod_results = compare_ds(model, 
#                                  [prep.basic_loader(), coco_mod, coco_mod_histeq, coco_mod_retinex],
#                                  ['Original', 'Low Contrast', 'Low Contrast + HistEQ', 'Low Contrast + Retinex'], 
#                                  preview_idxs,
#                                  view_only=False)